<a href="https://colab.research.google.com/github/SimonSanfeliu/Algoritmos-y-Estructuras-de-Datos/blob/master/Tarea_4_Sim%C3%B3n_Sanfeli%C3%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC3001 Otoño 2020 Tarea 4
# Generación de código

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 8 de junio de 2020 a las 23:59

El objetivo de esta tarea es aprender sobre la manipulación de árboles que representan fórmulas. Hay muchas cosas que uno puede hacer con estos árboles, y en esta tarea nos enfocaremos en particular en algo que los compiladores hacen: la **generación de código** para evaluar fórmulas.

Específicamente, el problema es: dada una fórmula, generar una secuencia de instrucciones elementales que la evalúen. Por ejemplo, dada una fórmula como

```
(a+b)*(c-d)
```

se podría generar código como el siguiente:

```
t1=a+b
t2=c-d
t3=t1*t2
```

Las instrucciones elementales son siempre de la forma ``ti=x op y``, donde ``ti`` es una variable temporal, ``x`` e ``y`` pueden ser variables o números, y ``op`` es un operador. Uno va creando variables ``t1``, ``t2``, ... a medida que las necesita, y la última de ellas es la que almacena el valor final de la fórmula.

Las fórmulas pueden ser arbitrariamente complicadas, como por ejemplo la fórmula

$$
(2-p\,q)\left( \frac{1}{n}+\frac{1}{p+q}\right)
$$

la cual escribimos como

```
(2-p*q)*(1/n+1/(p+q))
```

y cuyo código generado podría ser,

```
t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6
```

El objetivo de esta tarea es que usted implemente una función que, dado un árbol ya construído, lo recorra para generar una secuencia de instrucciones elementales que lo evalúe.

A continuación usted encontrará clases ``Nodoi``, ``Nodoe`` y ``Arbol`` que implementan árboles que representan fórmulas. La clase ``Arbol`` viene con un constructor que recibe un string como parámetro, el cual contiene una fórmula, y este constructor se encarga de transformar la fórmula de string a árbol, de modo que usted no necesita preocuparse de eso. Para simplificar, la fórmula solo podrá contener variables de una sola letra, números de un solo dígito, y sin espacios. Los operadores permitidos son solo suma, resta, multiplicación y división. No hay operador menos unario.

Usted tienen que implementar para la clase ``Arbol`` una función ``codigo`` que al ser invocado genere una secuencia de instrucciones elementales para evaluar la fórmula representada por ese árbol, y la retorne en forma de una lista (de Python).

In [0]:
oper = '+-*/' #String con los operadores permitidos para la fórmula.

class Nodoi:
    def __init__(self, izq, info, der):
        global l #Se agrega una lista global en donde se guardará la fórmula ingresada reescrita en postorden.
        l = []
        self.izq=izq
        self.info=info
        self.der=der
    def postorden(self):
        self.izq.postorden()   
        self.der.postorden()
        print(self.info, end=" ")
    def listapostorden(self): #Función que irá ingresando la fórmula en postorden a la lista global definida anteriormente.
        global l
        self.izq.listapostorden()
        self.der.listapostorden()
        l.append(self.info)

class Nodoe:
    def __init__(self, info):
        self.info=info
    def postorden(self):
        print(self.info, end=" ")
    def listapostorden(self): #Función que ingresa el valor del nodo a la lista global con la fórmula en postorden.
        global l
        l.append(self.info)

class Arbol:
    def __init__(self,formula):
        global k
        global s
        s=formula+";" # agregamos una marca de fin de la entrada
        k=0 # indica próximo caracter por procesar
        # definimos funciones para analizar la fórmula
        def expresion(): # retorna puntero a la raíz de un árbol que representa a la fórmula s
            global k
            global s
            a=factor()
            while s[k]=="+" or s[k]=="-":
                op=s[k]
                k+=1
                b=factor()
                a=Nodoi(a,op,b)
            return a
        def factor():
            global k
            global s
            a=termino()
            while s[k]=="*" or s[k]=="/":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        def termino(): # posible constante, variable o formula parentizada
            global k
            global s
            if s[k].isalpha() or s[k].isdigit():
                a=Nodoe(s[k])
                k+=1
                return a
            if s[k]=="(": # fórmula parentizada
                k+=1
                a=expresion()
                if s[k]!=")":
                    print("Error: Falta cierra paréntesis: "+formula[k:])
                    assert False
                k+=1
                return a
            print("Error: Falta variable, número o abre paréntesis: "+formula[k:])
            assert False
              
        a=expresion()
        if s[k]!=";":
            print("Error: Basura al final de la fórmula: "+formula[k:])
            assert False                
        self.raiz=a
            
    def postorden(self):
        print("Postorden:", end=" ")
        self.raiz.postorden()
        print()

    #Función que entrega una lista con la fórmula ordenada en postorden.
    def listapostorden(self):
        global l
        if type(self.raiz) == Nodoe: #Si es que la fórmula es sólo un término, el árbol será sólo un nodo externo, 
          l = [self.raiz.info]       #lo que hace que la lista tenga sólo ese término.
          return l
        self.raiz.listapostorden()
        return l 
    
    def codigo(self):
        lt = [] #Lista temporal donde se crean las variables temporales con las instrucciones.
        lf = [] #Lista final con las instrucciones pedidas.
        i = 1   #Contador para enumerar las variables temporales.
        L = self.listapostorden() #Lista de la fórmula en postorden.
        if len(L) == 1:     #Si se ingresó sólo una variable o número, la única instrucción es retornar ésta misma,
          lf = ['t1='+L[0]] #por esto mismo es que la lista final queda de un sólo término, que es este.
          return lf
        else:
          for x in L:
            if x.isdigit() or x.isalpha(): #Si es un número o una letra, se agrega a la lista temporal.
              lt.append(x)
            elif x in oper: #Si es un operador, necesariamente hay como mínimo dos términos en la lista lt, que equivalen a las hojas
              n = len(lt)   #hijas de dicho operador.
              k1 = lt.pop(n-1) #Se obtienen los valores de las hojas hijas.
              k2 = lt.pop(n-2)
              p = k2 + x + k1 #Se crea la operación correspondiente.
              v = 't'+str(i) + '=' + p #Se crea la variable temporal y se guarda en la listas de instrucciones.
              lf.append(v)
              lt.append('t'+str(i)) #Se agrega la variable temporal a la lista temporal para que las siguientes variables queden en función de ésta.
              i += 1 #Se debe ocupar una nueva variable temporal para los siguientes términos.
          return lf #Se entrega la lista con las instrucciones.

A continuación probamos esta funcionalidad con varias fórmulas de ejemplo. Para ver si el árbol fue construido correctamente, lo recorremos en postorden para imprimirlo en notación polaca. **Importante**: La notación polaca no tiene nada que ver con lo que ustedes tienen que hacer, es solo una manera de poder imprimir el árbol generado para chequear que esté bien construido.

In [0]:
Arbol("a*b*c").postorden()

Postorden: a b * c * 


In [0]:
Arbol("(a+b)*(c-d)").postorden()

Postorden: a b + c d - * 


In [0]:
Arbol("(2-p*q)*(1/n+1/(p+q))").postorden()

Postorden: 2 p q * - 1 n / 1 p q + / + * 


## Resultados

En esta sección probamos la generación de código que usted tiene que escribir. Mientras usted no sustituya la función ``codigo`` provisoria que viene en la definición de la clase, va a aparecer la lista que generamos ahí. Primero vamos a definir una función ``probar`` que genera el código a partir de una fórmula dada e imprime el resultado:

In [0]:
def probar(formula):
    a=Arbol(formula)
    for x in a.codigo():
        print(x)

A continuación probamos con varias fórmulas. Además de las fórmulas dadaas, usted tiene que agregar tres ejemplos adicionales.

In [0]:
probar("a+1")

t1=a+1


In [0]:
probar("(a+b)*(c-d)")

t1=a+b
t2=c-d
t3=t1*t2


In [0]:
probar("(2-p*q)*(1/n+1/(p+q))")

t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6


In [0]:
probar("((((a+b)+c)+d)+e)")

t1=a+b
t2=t1+c
t3=t2+d
t4=t3+e


In [0]:
probar("(a+(b+(c+(d+e))))")

t1=d+e
t2=c+t1
t3=b+t2
t4=a+t3


In [0]:
probar("a")

t1=a


## Discusión

Suponga que hay escasez de variables temporales, así que se le ha pedido que trate de reciclarlas lo más posible, y solo crear nuevas variables ``ti`` cuando sea estrictamente necesario. Por ejemplo, para la fórmula

```
((a+b)*c)+((d-e)/f)
```

si generamos código como lo hemos hecho en los ejemplos anteriores (código de la izquierda), se usan 5 variables temporales, pero el código de la derecha muestra que se puede evaluar usando solo 2:

```
t1=a+b               t1=a+b
t2=t1+c              t1=t1*c
t3=d-e               t2=d-e
t4=t3/f              t2=t2/f
t5=t2+t4             t1=t1+t2
```

Discuta cómo se podría modificar su algoritmo para que (1) use menos variables temporales, y luego (2) para que use el mínimo posible de variables temporales.

No es necesario que implemente nada, solo que discuta cómo se podría hacer.

Dado que la lista temporal usada en la función de código es utilizada con las mismas características que las de una pila, para ser más eficiente con el tiempo y con la operatoria, podría modificarse el algoritmo para que se utilice este tipo de estructuras en vez de la lista. Esto se haría recorriendo el árbol nuevamente en postorden, metiendo todos los elementos distintos de operadores a la pila. Una vez que se llegue a un operador en la fórmula, se hacen dos pop.() a la pila, se crea la instrucción y se le nombra con la varable temporal correspondiente. Luego de esto, la instrucción se guarda en la lista pedida y la variable se guarda en la pila utilizada. Es decir, es exactamente el mismo proceso pero con un mejor manejo del tiempo de ejecución y una mayor eficiencia respecto a la memoria. 

Ahora bien, ésto también permitiría una edición más simple en el código para reutilizar variables temporales. El proceso estaría enfocado en el recorrer el árbol en postorden, de igual manera que la forma original, pero si es que se encuentran dos operadores seguidos en ésta antes de llegar a la primera raíz (al final de la fórmula en postorden) entonces se puede editar una variable temporal para su reutilización. Esto se haría siguiendo la lógica de que, si es los operadores están seguidos es porque uno de ellos es una hoja hija del otro, por lo que primero debe operarse la hija y luego este resultado con el padre. Esto puede lograrse realizando la operación hija y luego operar ésta con la operación padre pero guardándola en la misma variable dentro de la pila. Por ejemplo, si se tiene (2-(p+q)), que en postorden se vería como "2 p q + -", en vez de hacer 

*   t1 = p+q
*   t2 = 2-t1

Al notar que hay dos operadores seguidos haría

*   t1 = p+q
*   t1 = 2-t1

Esto es escencial a la hora de reemplazar las variables algebraicas por su valor numérico, ya que así se puede seguir una línea coherente al correcto uso de los operadores matemáticos. 

En el caso en el que haya sólo un operador, si es que es hoja hermana es un nodo interno igual que él, entonces sólo se creará una variable que igualmente se guardará en la pila, la que podrá ser operada correctamente una vez que se junte con las demás siguiendo este mismo método. En caso de que la hoja hermana del operador sea un nodo externo, entonces la primera operatoria que involucra a este nodo externo puede ser guardada en la misma variable que la del operador solitario.

Dado esto, y asumiendo que idealmente se deben priorizar las variables con un subíndice menor aunque el resultado sea el mismo, debe también agregarse un correcto uso del nombramiento de aquellas variables que junten otras variables de una raíz hermana. Para entendderlo mejor, si se tiene el caso (1/n+1/(p+q)), la operatoria con variables reducidas, recorriendo el árbol en postorden, sería la siguiente:

*   t1 = 1/n
*   t2 = p+q
*   t2 = 1/t2
* t1 = t1+t2

Pero este resultado sería el mismo si es que se hubiera hecho en el último paso "t2 = t1+t2", ya que mientras se mantenga el orden y el lado de la operatoria, el resultado sigue siendo el mismo pero guardado en una variable distinta. Si es que llegase a ser estrictamente necesario que se guarde en la variable de índice menor, debe implementarse esta medida de igual forma al código. 

Ahora bien, con respecto a la cantidad mínima de variables que pueden obtenerse, depende meramente de la complejidad del árbol o fórmula en cuestión. Es fácil ver que, si es que el árbol se expande con nodos internos a un lado y nodos externos al otro, será necesaria solamente una variable temporal. Por ejemplo, para el caso (((a+b)+c)+d), escrito en postorden se ve que "a b + c + d +", viendo que los nodos internos son sólo hacia la izquierda, se tiene que

* t1 = a+b
* t1 = t1+c
* t1 = t1+d

Este proceso es idéntico si es que el árbol estuviese construido hacia la derecha, es decir si postorden fuera "d c a b + + + +", siguiendo la metodología propuesta anteriormente. 

Ahora, viendo otros ejemplos más complejos cuando hay más nodos internos a ambos lados, se ve que la cantidad de variables necesariamente aumentará; dado que ha aumentado la complejidad del árbol. Tomando ((a+b)*c)+((d-e)/f), escrito en postorden se puede ver que presenta dos nodos internos a cada lado; concordando que en el ejemplo dado utiliza dos variables temporales como mínimo. Por último, si se toma (2-p*q)*(1/n+1/(p+q)) y se revisa la composición de su árbol, se podrá observar que en un lado hay 2 nodos internos y en el otro habrán 4. Viendo ahora su mínima cantidad de variables temporales

* t1 = p*q
* t1 = 2-t1
* t2 = 1/n
* t3 = p+q
* t3 = 1/t3
* t2 = t2+t3
* t1 = t1*t2

En total, se tienen 3 variables temporales. Por ende, podría decirse que la mínima cantidad de variables temporales está dada por la cantidad de nodos internos y su distribución. En concreto, para mantener el mínimo, se creará una variable temporal cada vez que haya una secuencia de nodos internos a un lado de la raíz y que tenga al otro lado un nodo externo. Si es que un nodo interno tiene nodos internos a ambos lados, entonces se deberá crear otra variable temporal para poder hacer correctamente la operatoria. Esto queda más claro en el siguiente gráfico del árbol del último ejemplo:

![Árbol binario con sus respectivas variables temporales reutilizables.](https://drive.google.com/file/d/1pVoDTg-VoyyJPvBpsWQsUdErJ9zoMszA/view?usp=sharing)

Aquí se puede ver cómo son las mínimas variables reutilizables, siendo que para unir con los últimos operadores se reemplazan entre ellas hasta quedar sólo una; pero sieuge siendo necesario la creación de 3 para operar correctamente.




